## Coordinate Frames

| Frame | Units         | Origin   | Y dir | Description |
| ----- | ------------- | -------- | ----- | ----------- |
| qwn   | window pixels | top left | down  | Qt window coordinates |
| ndc   | half windows  | center   | up    | OpenGL normalized device coordinates |
| nic   | half images   | center   | up    | vimage normalized reoriented image coordinates |
| omp   | image pixels  | top left | down  | EXIF reoriented image pixels |
| txc   | images        | top left | down  | OpenGL texture coordinates |
| ypr   | degrees       | center   | up    | yaw and pitch of 360 image |

### Situations where transformation between coordinate frames is needed:

 1. When single dragging images to pan. Here we need the jacobian matrix relating qwn coordinates to view_model parameters. $viw_J_qwn$
     * image center (but not zoom), in the case of rectangular images
     * view pitch and heading angles (but not zoom nor roll) in the case of 360 images
 2. When rendering images in a GLSL shader. $txc_X_ndc$
 3. When hovering with the mouse $omp_X_qwn$ for rectangular and $viw_X_qwn$ for 360 images.

The only frames we need to compute are:
  * tex, in the rectangular shader tex_X_ndc
  * omp, on hover omp_X_qwn
  * derivative of image center (in either ont, raw, tex, img, or omp), on drag omp_J_qwn
    * ypr for 360s ypr_J_qwn
    * omp for rectangular omp_J_qwn
  
Rectangular view state:
  * image center (should be omp, TODO:)
  * zoom (windows per image)
360 view state:
  * pitch, yaw (maybe as equirect image center in degrees)
  * zoom (windows per image at center?)
  
What are normalized image coordinates (nic) ?
  * origin (0,0) is at image center
  * Y increases upward
  * EXIF reorientation is applied
  * coordinates range from -1 (left, bottom) to +1 (right, top)
  * scaled such that underlying full image exactly fits in the window. The edge touching dimension ranges from -1 to +1
  
Vertex shader converts from ndc to nic
  * apply zoom, window aspect, image size, image center
Fragment shader computes from nic to txc, and samples
  * apply pixel filter
  * 360 images:
    * nic to geocentric unit sphere
    * rotate by yaw pitch roll
    * project to projection...


In [1]:
from sympy import *
init_printing()

The transform required for hover coordinates is composed of sub-transforms:

$^{omp}p \ = \ ^{omp}X^{qwn} \ \cdot \ ^{qwn}p $

$^{omp}X^{qwn} \ = \ ^{omp}X^{nic} \ \cdot \ ^{nic}X^{ndc} \ \cdot \ ^{ndc}X^{qwn}$


In [ ]:
# printed symbols of transforms
omp_X_qwn_s = symbols("^{omp}X^{qwn}")
ndc_X_qwn_s = symbols("^{ndc}X^{qwn}")
nic_X_ndc_s = symbols("^{nic}X^{ndc}")
omp_X_nic_s = symbols("^{omp}X^{nic}")

In [54]:
# ndc_X_qwn: Transform to OpenGL normalized device coordinates from Qt window pixel coordinates

# window dimensions
w_win, h_win = symbols("w_win, h_win")

ndc_X_qwn_m = Matrix([
    [2/w_win, 0, -1],
    [0, -2/h_win, 1],
    [0, 0, 1],
])

# confidence checks

# Upper left corner
ulc_qwn_v = Matrix([0, 0, 1])
ulc_ndc_v = Matrix([-1, 1, 1])
assert ndc_X_qwn_m * ulc_qwn_v == ulc_ndc_v

# Upper right corner
urc_qwn_v = Matrix([w_win, 0, 1])
urc_ndc_v = Matrix([1, 1, 1])
assert ndc_X_qwn_m * urc_qwn_v == urc_ndc_v

# Lower left corner
llc_qwn_v = Matrix([0, h_win, 1])
llc_ndc_v = Matrix([-1, -1, 1])
assert ndc_X_qwn_m * llc_qwn_v == llc_ndc_v

# Lower right corner
lrc_qwn_v = Matrix([w_win, h_win, 1])
lrc_ndc_v = Matrix([1, -1, 1])
assert ndc_X_qwn_m * lrc_qwn_v == lrc_ndc_v

Eq(ndc_X_qwn_s, ndc_X_qwn_m, evaluate=False)

                 ⎡  2             ⎤
                 ⎢─────    0    -1⎥
                 ⎢w_win           ⎥
                 ⎢                ⎥
^{ndc}X__{qwn} = ⎢        -2      ⎥
                 ⎢  0    ─────  1 ⎥
                 ⎢       h_win    ⎥
                 ⎢                ⎥
                 ⎣  0      0    1 ⎦

In [67]:
# nic_X_ndc: transform to vimage normalized image coordinates from OpenGL normalized device coordinates

w_omp, h_omp = symbols("w_omp, h_omp")  # (reoriented) image dimensions
zoom = symbols("zoom")  # windows per image (default 1)
cen_x_omp = symbols("^{omp}cen_x")

# TODO: aspect, center
aspect_omp = w_omp / h_omp
aspect_qwn = w_win / h_win

# If aspect_omp > aspect_qwn, image is fatter than window, so pad top/bottom at zoom == 1
#   -> x/y aspect scales are [w_omp / w_omp, h_omp/w_omp] == [1, h_omp/w_omp]
# otherwise, pad left/right at zoom == 1
#   -> x/y aspect scales are [w_omp / h_omp, h_omp/h_omp] == [h_omp/w_omp, 1]
rc_scaleA = w_win
rc_scaleB = h_win

nic_X_ndc_mA = Matrix([
    [w_win/rc_scaleA/zoom, 0, 0],
    [0, -h_win/rc_scaleA/zoom, 0],
    [0, 0, 1],
])

Eq(nic_X_ndc_s, nic_X_ndc_mA, evaluate=False)

                 ⎡ 1                 ⎤
                 ⎢────      0       0⎥
                 ⎢zoom               ⎥
                 ⎢                   ⎥
^{nic}X__{ndc} = ⎢       -h_win      ⎥
                 ⎢ 0    ──────────  0⎥
                 ⎢      w_win⋅zoom   ⎥
                 ⎢                   ⎥
                 ⎣ 0        0       1⎦

In [68]:
nic_X_ndc_mB = Matrix([
    [w_win/rc_scaleB/zoom, 0, 0],
    [0, -h_win/rc_scaleB/zoom, 0],
    [0, 0, 1],
])

Eq(nic_X_ndc_s, nic_X_ndc_mB, evaluate=False)

                 ⎡  w_win            ⎤
                 ⎢──────────   0    0⎥
                 ⎢h_win⋅zoom         ⎥
                 ⎢                   ⎥
^{nic}X__{ndc} = ⎢            -1     ⎥
                 ⎢    0       ────  0⎥
                 ⎢            zoom   ⎥
                 ⎢                   ⎥
                 ⎣    0        0    1⎦

In [70]:
# omp_X_nic: Transform from normalized image coordinates to oriented image pixels

# TODO:
omp_X_nic_mA = Matrix([
    [w_omp, 0, w_omp / 2],
    [0, -w_omp, h_omp / 2],
    [0, 0, 1],
])

# nic_X_ndc_mB * ndc_X_qwn_m

⎡    2                    -w_win   ⎤
⎢──────────      0       ──────────⎥
⎢h_win⋅zoom              h_win⋅zoom⎥
⎢                                  ⎥
⎢                2          -1     ⎥
⎢    0       ──────────     ────   ⎥
⎢            h_win⋅zoom     zoom   ⎥
⎢                                  ⎥
⎣    0           0           1     ⎦

### Qt window coordinates (qwn) <-> centered window coordinates (cwn)

In [2]:
# Convert between Qt window pixel coordinates (qwn) and centered window pixel coordinates (cwn)
x_qwn, y_qwn = symbols("^{qwn}p_x, ^{qwn}p_y")  # Qt x,y window position in pixels
xyw_qwn = Matrix([x_qwn, y_qwn, 1])
width_win, height_win = symbols("width_win, height_win")  # Window size in pixels
cwn_X_qwn = Matrix([
    [1, 0, -width_win / 2],
    [0, -1, height_win / 2],
    [0, 0, 1],
])

# Verify assumptions
assert cwn_X_qwn * Matrix([0, 0, 1]) == Matrix([-width_win/2, height_win/2, 1])  # upper left
assert cwn_X_qwn * Matrix([width_win, height_win, 1]) == Matrix([width_win/2, -height_win/2, 1]) # lower right
assert cwn_X_qwn * Matrix([width_win/2, height_win/2, 1]) == Matrix([0, 0, 1]) # center

x_cwn, y_cwn = symbols("^{cwn}p_x, ^{cwn}p_y")
xyw_cwn = Matrix([x_cwn, y_cwn, 1])
UnevaluatedExpr(cwn_X_qwn) * UnevaluatedExpr(xyw_qwn.transpose())
rhs = UnevaluatedExpr(cwn_X_qwn) * UnevaluatedExpr(xyw_qwn.transpose())
Eq(UnevaluatedExpr(xyw_cwn), rhs)

              ⎡       -width_win ⎤                         
              ⎢1  0   ───────────⎥                         
⎡^{cwn}pₓ ⎤   ⎢            2     ⎥                         
⎢         ⎥   ⎢                  ⎥                         
⎢^{cwn}p_y⎥ = ⎢       height_win ⎥⋅[^{qwn}pₓ  ^{qwn}p_y  1]
⎢         ⎥   ⎢0  -1  ────────── ⎥                         
⎣    1    ⎦   ⎢           2      ⎥                         
              ⎢                  ⎥                         
              ⎣0  0        1     ⎦                         

In [3]:
# Inverse transform
qwn_X_cwn = cwn_X_qwn.inv()

# Verify assumptions
assert qwn_X_cwn * Matrix([-width_win/2, height_win/2, 1]) == Matrix([0, 0, 1])  # upper left
assert qwn_X_cwn * Matrix([width_win/2, -height_win/2, 1]) == Matrix([width_win, height_win, 1]) # lower right
assert qwn_X_cwn * Matrix([0, 0, 1]) == Matrix([width_win/2, height_win/2, 1]) # center

rhs = UnevaluatedExpr(qwn_X_cwn) * UnevaluatedExpr(xyw_cwn.transpose())
Eq(UnevaluatedExpr(xyw_qwn), rhs)

              ⎡       width_win ⎤                         
              ⎢1  0   ───────── ⎥                         
⎡^{qwn}pₓ ⎤   ⎢           2     ⎥                         
⎢         ⎥   ⎢                 ⎥                         
⎢^{qwn}p_y⎥ = ⎢       height_win⎥⋅[^{cwn}pₓ  ^{cwn}p_y  1]
⎢         ⎥   ⎢0  -1  ──────────⎥                         
⎣    1    ⎦   ⎢           2     ⎥                         
              ⎢                 ⎥                         
              ⎣0  0       1     ⎦                         

In [4]:
# Jacobian
dx_qwn, dy_qwn = symbols("^{qwn}{\partial}p_x, ^{qwn}{\partial}p_y")
dx_cwn, dy_cwn = symbols("^{cwn}{\partial}p_x, ^{cwn}{\partial}p_y")
cwn_J_qwn = cwn_X_qwn[0:2,0:2]
Eq(UnevaluatedExpr(Matrix([dx_cwn, dy_cwn])), UnevaluatedExpr(cwn_J_qwn) * UnevaluatedExpr(Matrix([dx_qwn, dy_qwn]).transpose()))
# cwn_J_qwn

⎡^{cwn}{\partial}pₓ ⎤   ⎡1  0 ⎤                                          
⎢                   ⎥ = ⎢     ⎥⋅[^{qwn}{\partial}pₓ  ^{qwn}{\partial}p_y]
⎣^{cwn}{\partial}p_y⎦   ⎣0  -1⎦                                          

In [5]:
qwn_J_cwn = qwn_X_cwn[0:2,0:2]
assert qwn_J_cwn * cwn_J_qwn == eye(2)
Eq(UnevaluatedExpr(Matrix([dx_qwn, dy_qwn])), UnevaluatedExpr(qwn_J_cwn) * UnevaluatedExpr(Matrix([dx_cwn, dy_cwn]).transpose()))


⎡^{qwn}{\partial}pₓ ⎤   ⎡1  0 ⎤                                          
⎢                   ⎥ = ⎢     ⎥⋅[^{cwn}{\partial}pₓ  ^{cwn}{\partial}p_y]
⎣^{qwn}{\partial}p_y⎦   ⎣0  -1⎦                                          